In [1]:
import pandas as pd
import os

In [5]:
file_path = os.path.join('DataCollected', 'final_new.xlsx')
print(file_path)
df = pd.read_excel(file_path)

DataCollected\final_new.xlsx


In [92]:
file_path = os.path.join('DataBase', 'db.xlsx')
df = pd.read_excel(file_path, sheet_name="DS Hội đồng_DN", skiprows=2)
df = df.drop(columns = ['Unnamed: 4'])
df['Đơn vị.1']

0     Khoa Kỹ thuật Điện tử 1, Học viện Công nghệ Bư...
1     Khoa Kỹ thuật Điện tử 1, Học viện Công nghệ Bư...
2     Khoa Kỹ thuật Điện tử 1, Học viện Công nghệ Bư...
3     Tổng công ty Công nghiệp Công nghệ cao, Tập đo...
4     Khoa Kỹ thuật Điện tử 1, Học viện Công nghệ Bư...
5                                                   NaN
6                                                   NaN
7                                                   NaN
8                                                   NaN
9     Khoa Kỹ thuật Điện tử 1, Học viện Công nghệ Bư...
10    Khoa Kỹ thuật Điện tử 1, Học viện Công nghệ Bư...
11    Khoa Kỹ thuật Điện tử 1, Học viện Công nghệ Bư...
12           Trung tâm Kỹ thuật, Đài Tiếng nói Việt Nam
13    Khoa Kỹ thuật Điện tử 1, Học viện Công nghệ Bư...
14                                                  NaN
15                                                  NaN
16                                                  NaN
17                                              

In [6]:
def get_all_councils():
    file_path = os.path.join('DataBase', 'db.xlsx')
    df = pd.read_excel(file_path, sheet_name="DS Hội đồng_DN", skiprows=2)
    df = df.drop(columns = ['Unnamed: 4'])
    df = df.dropna(ignore_index=True)
    df = df[df['Họ và tên'] != 'Họ và tên']
    councils = {}
    for i in range(len(df)):
        member = df.iloc[i]
        name = member['Họ và tên']
        role = member['Nhiệm vụ'].replace('\xa0', '')
        abbr_unit = member['Đơn vị']
        unit = member['Đơn vị.1']
        id_council = f"HD{(i)//5 + 1}"
        if councils.get(id_council) is None:
            councils[id_council] = [{
                'name': name,
                'role': role,
                'abbr_unit': abbr_unit,
                'unit': unit
            }]
        else:
            councils[id_council].append({
                'name': name,
                'role': role,
                'abbr_unit': abbr_unit,
                'unit': unit
            })
    return councils

In [94]:
# def find_student_by_council_and_group_id(council_id=None, group_id=None):
#     students = []
#     if group_id is None or council_id is None:
#         return students
#     file_path = os.path.join('DataBase', 'db.xlsx')
#     df2 = pd.read_excel(file_path, sheet_name="DS SV các Hội đồng phân PB", skiprows=1)
#     df2 = df2.drop(columns = ['Unnamed: 0'])
#     df2.loc[1, 'Unnamed: 11'] = df2.loc[0, 'Unnamed: 11']
#     df2.loc[1, 'Unnamed: 12'] = df2.loc[0, 'Unnamed: 12']
#     df2.loc[1, 'Unnamed: 3'] = 'Tên'
#     df2.columns = df2.iloc[1]
#     df2 = df2.dropna(how='all', ignore_index=True)
#     index_council = find_index_hd_in_excel(df2, council_id)
#     index_council_next = find_index_hd_in_excel(df2, council_id + 1)
#     if index_council_next == -1:
#         index_council_next = len(df2)
#     df2 = df2.iloc[index_council+2:index_council_next]
#     df2 = df2.reset_index(drop=True)
#     for i in range(len(df2)):
#         df2 = df2.reset_index(drop=True) 
#         if df2.loc[i, 'Nhóm'] == group_id:
#             students.append(df2.loc[i, 'Mã sinh viên'])
#     print("Students", students)
#     return students

# def find_index_hd_in_excel(df2, id_hd):
#     for i in range(len(df2)):
#         if df2.loc[i, 'Mã sinh viên'] == f'Hội đồng {id_hd}':
#             return i
#     return -1

In [11]:
def get_students_by_project_name(project_name):
    file_path = os.path.join('DataBase', 'db.xlsx')
    df = pd.read_excel(file_path, sheet_name="Danh sách các đồ án ", skiprows=12)
    df = df.ffill()

    students = []
    col = 'Tên đề tài đồ án/ khóa luận tốt nghiệp'
    df_students = df[df[col].str.contains(project_name, case=False, regex=False)]
    
    first_students = list(df_students['Họ và tên'])
    second_students = list(df_students['Unnamed: 3'])
    fullname_students = [f"{first} {second}" for first, second in zip(first_students, second_students)]
    msv_students = list(df_students['Mã sinh viên'])
    day_of_birth_students = list(df_students['Năm sinh'])
    class_students = list(df_students['Lớp'])
    tt_students = list(df_students['TT'])
    for i in range(len(fullname_students)):
        students.append({
            'fullname': fullname_students[i],
            'msv': msv_students[i],
            'day_of_birth': day_of_birth_students[i],
            'class': class_students[i],
            'tt': int( tt_students[i])
        })
    return students

In [29]:
file_path = os.path.join('DataBase', 'db.xlsx')
df2 = pd.read_excel(file_path, sheet_name="DS SV các Hội đồng phân PB", skiprows=1)
df2.loc[1, 'Unnamed: 0'] = 'TT'
df2.loc[1, 'Unnamed: 11'] = df2.loc[0, 'Unnamed: 11']
df2.loc[1, 'Unnamed: 12'] = df2.loc[0, 'Unnamed: 12']
df2.loc[1, 'Unnamed: 3'] = 'Tên'
df2.columns = df2.iloc[1]
df2 = df2.dropna(how='all', ignore_index=True)
df2

C:\Users\xin chao\AppData\Local\Temp\ipykernel_14152\1741099478.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'TT' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df2.loc[1, 'Unnamed: 0'] = 'TT'


1,TT,Mã sinh viên,Họ và tên,Tên,Năm sinh,Lớp,Giáo viên hướng dẫn,Đơn vị,Bộ môn,Tên đề tài đồ án/ khóa luận tốt nghiệp,Loại đồ án,Nhóm,Phản biện (Khoa)
0,NaN,Hội đồng 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nhóm,Phản biện (Khoa)
1,TT,Mã sinh viên,Họ và tên,Tên,Năm sinh,Lớp,Giáo viên hướng dẫn,Đơn vị,Bộ môn,Tên đề tài đồ án/ khóa luận tốt nghiệp,Loại đồ án,Nhóm,Phản biện (Khoa)
2,1.0,B20DCDT017,Phạm Thế,Anh,13/11/2002,D20DTMT1,TS. Nguyễn Trung Hiếu,Khoa KTĐT,ĐTMT,"Nghiên cứu, thiết kế thiết bị thu âm tiếng nói...",Đồ án cá nhân,1,ThS. Nguyễn Quốc Dinh
3,2.0,B20DCDT029,Trần Ngọc,Danh,27/11/2002,D20DTMT1,TS. Nguyễn Trung Hiếu,Khoa KTĐT,ĐTMT,Hệ thống cảnh báo tài xế ngủ gật trên oto,Đồ án cá nhân,2,ThS. Đinh Quang Ngọc
4,3.0,B20DCDT037,Nguyễn Tiến,Duy,09/10/2002,D20DTMT1,TS. Nguyễn Trung Hiếu,Khoa KTĐT,ĐTMT,Hệ thống hỗ trợ vận hành Wireless Audio Visual...,Đồ án cá nhân,1,ThS. Nguyễn Quốc Dinh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,11.0,B20DCDT156,Nguyễn Thị,Nguyệt,01/10/2002,D20DTMT2,PGS. TS Trương Cao Dũng,Khoa KTĐT,ĐTMT,Chuyển Đổi Ngôn Ngữ Thường Sang Ngôn Ngữ Ký Hi...,Đồ án cá nhân,2,TS. Trần Thị Thúy Hà
117,12.0,B20DCDT026,Trần Đình,Chiến,09/02/2002,D20DTRB,ThS. Đào Thanh Huyền,Khoa KTĐT,ĐTMT,Mô hình khoá cửa thông minh nhận diện khuôn mặ...,Đồ án nhóm,NaN,TS. Trần Thị Thúy Hà
118,13.0,B20DCDT055,Nguyễn Minh,Đức,21/11/2002,D20DTMT2,ThS. Đào Thanh Huyền,Khoa KTĐT,ĐTMT,Hệ thống báo cháy và chữa cháy tự động,Đồ án nhóm,NaN,TS. Phan Thị Lan
119,14.0,B20DCDT218,Trịnh,Trung,31/07/2002,D20DTMT1,ThS. Phạm Văn Sự,Khoa KTĐT,XLTH,"Tìm hiểu, xây dựng hệ thống khóa dựa trên RFID",Đồ án nhóm,NaN,ThS. Chu Văn Cường


In [31]:
import os
import pandas as pd

file_path = os.path.join('DataBase', 'db.xlsx')
df2 = pd.read_excel(file_path, sheet_name="DS SV các Hội đồng phân PB", skiprows=1)
# df2 = df2.drop(columns = ['Unnamed: 0'])  
df2.loc[1, 'Unnamed: 11'] = df2.loc[0, 'Unnamed: 11']
df2.loc[1, 'Unnamed: 12'] = df2.loc[0, 'Unnamed: 12']
df2.loc[1, 'Unnamed: 3'] = 'Tên'
df2.loc[1, 'Unnamed: 0'] = 'TT'
df2.columns = df2.iloc[1]
df2 = df2.dropna(how='all', ignore_index=True)
councils = get_all_councils()
id_council = 'HD1'
students = {
}
for i in range(len(df2)):
    msv = df2.loc[i, 'Mã sinh viên']
    if 'Hội đồng' in msv:
        tmp = msv.split(' ')
        id_council = f"HD{tmp[-1]}"
        continue
    if msv == 'Mã sinh viên':
        continue
    name = df2.loc[i, 'Họ và tên'] + ' ' + df2.loc[i, 'Tên']
    day_of_birth = df2.loc[i, 'Năm sinh']
    class_name = df2.loc[i, 'Lớp']
    instructor = df2.loc[i, 'Giáo viên hướng dẫn']
    subject = df2.loc[i, 'Bộ môn ']
    project = df2.loc[i, 'Tên đề tài đồ án/ khóa luận tốt nghiệp']
    project_type = df2.loc[i, 'Loại đồ án']
    tt = df2.loc[i, 'TT']
    group = id_council + ' - ' + str(df2.loc[i, 'Nhóm']) if not pd.isnull(df2.loc[i, 'Nhóm']) else ''
    opponent = df2.loc[i, 'Phản biện  (Khoa)'] if not pd.isna(df2.loc[i, 'Phản biện  (Khoa)']) else ''
    council = councils[id_council]
    students[msv] = {
        'name': name,
        'day_of_birth': day_of_birth,
        'class_name': class_name,
        'instructor': instructor,
        'subject': subject,
        'project': project,
        'project_type': project_type,
        'group': group,
        'opponent': opponent,
        'council': council,
        'msv': msv,
        'tt': int(tt)
    }
    # Get students from final sheet
new_students = students.copy()
for msv in students.keys():
    student = students[msv]
    if student['project'] == 'None':
        continue
    project_name = student['project']
    students_same_project = get_students_by_project_name(project_name)
    for std in students_same_project:
        msv_std = std['msv']
        if msv_std not in students.keys():
            fullname = std['fullname']
            day_of_birth = std['day_of_birth']
            class_name = std['class']


            new_students[msv_std] = students[msv].copy()
            new_students[msv_std]['msv'] = msv_std
            new_students[msv_std]['name'] = fullname
            new_students[msv_std]['day_of_birth'] = day_of_birth
            new_students[msv_std]['class_name'] = class_name


C:\Users\xin chao\AppData\Local\Temp\ipykernel_14152\635606347.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'TT' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df2.loc[1, 'Unnamed: 0'] = 'TT'


In [32]:
import json
with open('students.json', 'w', encoding='utf-8') as f:
    json.dump(new_students, f, ensure_ascii=False, indent=4)    

In [22]:
a = {
    'a': 1,
    'b': 2,
    'c': 3,
    'd': 4
}

list(a.values())

[1, 2, 3, 4]

In [4]:
import os
import pandas as pd
file_path = os.path.join('DataBase', 'db.xlsx')
df = pd.read_excel(file_path, sheet_name="Danh sách các đồ án ", skiprows=12)
df = df.fillna(method='ffill')
df

C:\Users\xin chao\AppData\Local\Temp\ipykernel_14152\1996155697.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


,TT,Mã sinh viên,Họ và tên,Unnamed: 3,Năm sinh,Lớp,Giáo viên hướng dẫn,Đơn vị,Bộ môn,Tên đề tài đồ án/ khóa luận tốt nghiệp,Làm đồ án/Học phần TTTN,Unnamed: 11,Corrected name
0,1.0,B20DCDT004,Hoàng Việt,An,06/10/2002,D20DTMT2,TS. Trịnh Trung Hiếu,Khoa KTĐT,TĐH,Xây dựng hệ thống điều khiển nhà thông minh bằ...,Đồ án cá nhân,NaN,NaN
1,2.0,B20DCDT005,Nguyễn Văn,An,10/11/2002,D20DTMT1,ThS. Nguyễn Quang Biên,Khoa KTĐT,ĐTMT,Xây dựng hệ thống xác thực dữ liệu sử dụng FPGA,Đồ án cá nhân,NaN,NaN
2,3.0,B20DCDT010,Hoàng Tuấn,Anh,26/03/2002,D20DTMT1,TS. Nguyễn Ngọc Minh,Khoa KTĐT,ĐTMT,Hệ thống nhận diện và theo dõi đối tượng,Đồ án cá nhân,NaN,NaN
3,4.0,B20DCDT011,Lưu Ngọc,Anh,12/02/2001,D20DTMT2,ThS. Trương Minh Đức,Khoa KTĐT,XLTH,"Thiết kế, thử nghiệm ổ cắm thông minh điều khi...",Đồ án cá nhân,NaN,NaN
4,5.0,B20DCDT015,Nguyễn Tuấn,Anh,19/09/2002,D20DTMT2,ThS. Phạm Văn Sự,Khoa KTĐT,XLTH,Thiết kế bộ khuyếch đại vì sai folded cascode,Đồ án cá nhân,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,106.0,B20DCDT083,Nguyễn Văn,Hùng,30/03/2002,D20DTMT2,TS. Chu Văn Bền,Khoa KTĐT,ĐTMT,Xây dựng hệ thống phân loại và quản lý sản phẩ...,Đồ án nhóm,NaN,NaN
185,106.0,B20DCDT210,Lê Đức,Thịnh,22/03/2002,D20DTMT1,TS. Chu Văn Bền,Khoa KTĐT,ĐTMT,Xây dựng hệ thống phân loại và quản lý sản phẩ...,Đồ án nhóm,NaN,NaN
186,107.0,B20DCDT200,Đinh Duy,Thành,26/11/2002,D20DTMT2,ThS. Bùi Thị Dân,Khoa KTĐT,ĐTMT,Xây dựng ứng dụng IOT: bộ điều khiển tập trung...,Đồ án nhóm,NaN,NaN
187,107.0,B20DCDT195,Nguyễn Trọng,Tuyên,19/05/2002,D20XLTH,ThS. Bùi Thị Dân,Khoa KTĐT,XLTH,Xây dựng ứng dụng IOT: bộ điều khiển tập trung...,Đồ án nhóm,NaN,NaN


In [2]:
import os
import pandas as pd

file_path = os.path.join('DataCollected', 'final_new.xlsx')
df = pd.read_excel(file_path)

In [3]:
df1 = df[df[' Mã sinh viên'] == 'B20DCDT158']
df1

,Họ và tên,Mã sinh viên,Lớp,HDCM_uv1-họ tên,HDCM_uv1_C3.3,HDCM_uv1_C4.2,HDCM_uv1_C5.3,HDCM_uv1_C6.3,HDCM_uv1_C6.4,HDCM_uv1_gpa,...,CBHD_3_C6.1,CBHD_3_C6.2,CBHD_3_gpa,CBPB-họ tên,CBPB_C2.3,CBPB_C3.2,CBPB_C4.1,CBPB_C6.1,CBPB_C6.2,CBPB_gpa
0,La Thị Hồng Nhung,B20DCDT158,D20XLTH,Phạm Hoàng Anh,8,8,8,8,8,8.0,...,9.0,10.0,9.2,Trương Minh Đức,7.0,8.0,7.0,8.0,7.0,7.4


In [4]:
grade_columns = [ 
        " HDCM_uv1_C3.3",   " HDCM_uv1_C4.2",   " HDCM_uv1_C5.3",   " HDCM_uv1_C6.3",   " HDCM_uv1_C6.4",   " HDCM_uv1_gpa",
        " HDCM_uv2_C3.3",   " HDCM_uv2_C4.2",   " HDCM_uv2_C5.3",   " HDCM_uv2_C6.3",   " HDCM_uv2_C6.4",   " HDCM_uv2_gpa",   
        " HDCM_uv3_C3.3",   " HDCM_uv3_C4.2",   " HDCM_uv3_C5.3",   " HDCM_uv3_C6.3",   " HDCM_uv3_C6.4",   " HDCM_uv3_gpa",   
        " HDCM_uv4_C3.3",   " HDCM_uv4_C4.2",   " HDCM_uv4_C5.3",   " HDCM_uv4_C6.3",   " HDCM_uv4_C6.4",   " HDCM_uv4_gpa",   
        " HDCM_uv5_C3.3",   " HDCM_uv5_C4.2",   " HDCM_uv5_C5.3",   " HDCM_uv5_C6.3",   " HDCM_uv5_C6.4",   " HDCM_uv5_gpa",   
        " CBHD_1_C1.1",   " CBHD_1_C1.2",   " CBHD_1_C5.1",   " CBHD_1_gpa",   
        " CBHD_2_C2.1",   " CBHD_2_C2.2",   " CBHD_2_C3.1",   " CBHD_2_C5.2",   " CBHD_2_gpa",   
        " CBHD_3_C2.3",   " CBHD_3_C3.2",   " CBHD_3_C4.1",   " CBHD_3_C6.1",   " CBHD_3_C6.2",   " CBHD_3_gpa",   
        " CBPB_C2.3",   " CBPB_C3.2",   " CBPB_C4.1",   " CBPB_C6.1",   " CBPB_C6.2",   " CBPB_gpa"
    ]

In [5]:
student = {}
for column in grade_columns:
    student[column] = df1[column].values[0] if not pd.isnull(df1[column].values[0]) else -1

In [7]:
student = {}
student['a'] = 1
student

{'a': 1}

In [10]:
import os
import pandas as pd
def getGradeOfStudent(msv, name):
    grade_student = {}
    file_path = os.path.join('DataCollected', 'final_new.xlsx')
    df = pd.read_excel(file_path)
    df_sv = df[df[' Mã sinh viên'] == msv]
    if df_sv.empty:
        return None
    prefix_column = []
    for column in df_sv.columns:
        if df.loc[0, column] == name:
            column_tmp = column.split('-')
            prefix_column.append(column_tmp[0])
    grade_columns = [ 
        " HDCM_uv1_C3.3",   " HDCM_uv1_C4.2",   " HDCM_uv1_C5.3",   " HDCM_uv1_C6.3",   " HDCM_uv1_C6.4",   " HDCM_uv1_gpa",
        " HDCM_uv2_C3.3",   " HDCM_uv2_C4.2",   " HDCM_uv2_C5.3",   " HDCM_uv2_C6.3",   " HDCM_uv2_C6.4",   " HDCM_uv2_gpa",   
        " HDCM_uv3_C3.3",   " HDCM_uv3_C4.2",   " HDCM_uv3_C5.3",   " HDCM_uv3_C6.3",   " HDCM_uv3_C6.4",   " HDCM_uv3_gpa",   
        " HDCM_uv4_C3.3",   " HDCM_uv4_C4.2",   " HDCM_uv4_C5.3",   " HDCM_uv4_C6.3",   " HDCM_uv4_C6.4",   " HDCM_uv4_gpa",   
        " HDCM_uv5_C3.3",   " HDCM_uv5_C4.2",   " HDCM_uv5_C5.3",   " HDCM_uv5_C6.3",   " HDCM_uv5_C6.4",   " HDCM_uv5_gpa",   
        " CBHD_1_C1.1",   " CBHD_1_C1.2",   " CBHD_1_C5.1",   " CBHD_1_gpa",   
        " CBHD_2_C2.1",   " CBHD_2_C2.2",   " CBHD_2_C3.1",   " CBHD_2_C5.2",   " CBHD_2_gpa",   
        " CBHD_3_C2.3",   " CBHD_3_C3.2",   " CBHD_3_C4.1",   " CBHD_3_C6.1",   " CBHD_3_C6.2",   " CBHD_3_gpa",   
        " CBPB_C2.3",   " CBPB_C3.2",   " CBPB_C4.1",   " CBPB_C6.1",   " CBPB_C6.2",   " CBPB_gpa"
    ]
    true_columns = []
    for column in grade_columns:
        for prefix in prefix_column:
            if column.startswith(prefix):
                true_columns.append(column)
    for column in true_columns:
        if not pd.isnull(df_sv[column].values[0]):
            grade_student[column.split('_')[-1]] = df_sv[column].values[0]
    return grade_student

In [9]:
getGradeOfStudent('B20DCDT190', 'Phạm Hoàng Anh')

[' HDCM', 'uv1']
[' HDCM', 'uv1']
[' HDCM', 'uv1']
[' HDCM', 'uv1']
[' HDCM', 'uv1']
[' HDCM', 'uv1']


{}

In [1]:
new_headers = ['Họ và tên', 'Mã sinh viên', 'Lớp', 
            'HDCM_uv1-họ tên', 'HDCM_uv1_C3.3', 'HDCM_uv1_C4.2', 'HDCM_uv1_C5.3', 'HDCM_uv1_C6.3', 'HDCM_uv1_C6.4', 'HDCM_uv1_gpa', 
            'HDCM_uv2-họ tên', 'HDCM_uv2_C3.3', 'HDCM_uv2_C4.2', 'HDCM_uv2_C5.3', 'HDCM_uv2_C6.3', 'HDCM_uv2_C6.4', 'HDCM_uv2_gpa', 
            'HDCM_uv3-họ tên', 'HDCM_uv3_C3.3', 'HDCM_uv3_C4.2', 'HDCM_uv3_C5.3', 'HDCM_uv3_C6.3', 'HDCM_uv3_C6.4', 'HDCM_uv3_gpa', 
            'HDCM_uv4-họ tên', 'HDCM_uv4_C3.3', 'HDCM_uv4_C4.2', 'HDCM_uv4_C5.3', 'HDCM_uv4_C6.3', 'HDCM_uv4_C6.4', 'HDCM_uv4_gpa', 
            'HDCM_uv5-họ tên', 'HDCM_uv5_C3.3', 'HDCM_uv5_C4.2', 'HDCM_uv5_C5.3', 'HDCM_uv5_C6.3', 'HDCM_uv5_C6.4', 'HDCM_uv5_gpa', 
            'CBHD_1-họ tên', 'CBHD_1_C1.1', 'CBHD_1_C1.2', 'CBHD_1_C5.1', 'CBHD_1_gpa', 
            'CBHD_2-họ tên', 'CBHD_2_C2.1', 'CBHD_2_C2.2', 'CBHD_2_C3.1', 'CBHD_2_C5.2', 'CBHD_2_gpa', 
            'CBHD_3-họ tên', 'CBHD_3_C2.3', 'CBHD_3_C3.2', 'CBHD_3_C4.1', 'CBHD_3_C6.1', 'CBHD_3_C6.2', 'CBHD_3_gpa', 
            'CBPB-họ tên', 'CBPB_C2.3', 'CBPB_C3.2', 'CBPB_C4.1', 'CBPB_C6.1', 'CBPB_C6.2', 'CBPB_gpa'
            ]

import os
import pandas as pd

file_path = os.path.join('DataCollected', 'final_new.xlsx')
df = pd.read_excel(file_path)
df.columns = new_headers
df.to_excel(file_path, index=False)